<img src = "https://learnopencv.com/wp-content/uploads/2024/09/Feature-Multimodal-RAG-with-ColPali-Gemini.gif">

In [ ]:
!pip install mteb tqdm typer seaborn -q
!pip install transformers  -U
!pip install pdf2image einops google-generativeai gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.6.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.8 MB/s eta 0:00:00
  Attempting

In [ ]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (336 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 123632 

### Installing Colpali

In [ ]:
!pip install git+https://github.com/illuin-tech/colpali -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.2 MB/s eta 0:00:00


In [ ]:
from typing import cast

import torch
from PIL import Image

from colpali_engine.models import ColPali, ColPaliProcessor

import os
import getpass
import gradio as gr
import torch
from pdf2image import convert_from_path
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm

import torch
from PIL import Image

### ColPali

1. Offline Indexing - ColPali

In [ ]:
model_name = "vidore/colpali-v1.3"

model = ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda",  # or "mps" if on Apple Silicon
).eval()

processor = ColPaliProcessor.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/751 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/78.6M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [ ]:
def index(file, ds):

    images = []
    for f in file:
        images.extend(convert_from_path(f))

    # run inference - docs
    dataloader = DataLoader(
        images,
        batch_size=4,
        shuffle=False,
        collate_fn=lambda x: processor.process_images(x),
    )
    for batch_doc in tqdm(dataloader):
        with torch.no_grad():
            batch_doc = {k: v.to("cuda") for k, v in batch_doc.items()}
            embeddings_doc = model(**batch_doc)
        ds.extend(list(torch.unbind(embeddings_doc.to("cuda"))))
    return f"Uploaded and converted {len(images)} pages", ds, images

In [ ]:
def search(queries: str, ds, images):
    qs = []
    with torch.no_grad():
        batch_queries = processor.process_queries([queries]).to(model.device)
        batch_queries = {k: v.to("cuda") for k, v in batch_queries.items()}
        queries_embeddings = model(**batch_queries)
        qs.extend(list(torch.unbind(queries_embeddings.to("cuda"))))

    # run evaluation

    scores = processor.score_multi_vector(qs, ds)
    best_page = int(scores.argmax(axis=1).item())
    return f"The most relevant page is {best_page}", images[best_page]

### Google Gemini API - For Generation

In [ ]:
import google.generativeai as genai

generation_config = {
  "temperature": 0.0,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 256,
  "response_mime_type": "text/plain",
}

genai.configure(api_key="GEMINI_API_KEY")

gemini_flash = genai.GenerativeModel(model_name="gemini-2.0-flash-exp" , generation_config=generation_config)

def get_answer(prompt:str , image:Image):
  response = gemini_flash.generate_content([prompt, image])
  return response.text

gemini_flash

genai.GenerativeModel(
    model_name='models/gemini-2.0-flash-exp',
    generation_config={'temperature': 0.0, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 1024, 'response_mime_type': 'text/plain'},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

### Test Gemini - Whether API Works

In [ ]:
!wget https://learnopencv.com/wp-content/uploads/2016/11/image-classification-pipeline.jpg -O ./cat.jpg -q


In [ ]:
#sanity check
image = Image.open("cat.jpg")
response = get_answer("Hello", image)
print(response)

### Gradio

Question that I tried that worked without any flaw:

Q1: Airtel Inflight Roaming packs

Q2: Jio Hotstar plans

Q3: Jio Entertainment plans

Q4: **Provide me details about Rs.739 Entertainment Plans of Jio**

Q5: **Provide me details about Rs.₹489 plan of Airtel**

In [ ]:
COLORS = ["#4285f4", "#db4437", "#f4b400", "#0f9d58", "#e48ef1"]

mock_image = Image.new("RGB", (448, 448), (255, 255, 255))

with gr.Blocks() as demo:
    gr.Markdown("# ColPali: Efficient Document Retrieval with Vision Language Models 📚🔍")
    gr.Markdown("## 1️⃣ Upload PDFs")
    file = gr.File(file_count="multiple")

    gr.Markdown("## 2️⃣ Index the PDFs and upload")
    convert_button = gr.Button("🔄 Convert and upload")
    message = gr.Textbox("Files not yet uploaded")
    embeds = gr.State(value=[])
    imgs = gr.State(value=[])

    # Define the actions for conversion
    convert_button.click(index, inputs=[file, embeds], outputs=[message, embeds, imgs])

    gr.Markdown("## 3️⃣ Search")
    query = gr.Textbox(placeholder="Enter your query to match")
    search_button = gr.Button("🔍 Search")

    gr.Markdown("## 4️⃣ ColPali Retrieval")
    message2 = gr.Textbox("Most relevant image is...")
    output_img = gr.Image()

    gr.Markdown("## 5️⃣ Gemini Response")
    output_text = gr.Textbox("Gemini Response...")

    def get_answer(user_prompt:str , image:Image):
       response = gemini_flash.generate_content([user_prompt, image])
       return response.text

    # Function to combine retrieval and LLM call
    def search_with_llm(query, ds, images):
        # Step 1: Search the best image based on query
        search_message, best_image = search(query, ds, images)

        # Step 2: Generate an answer using LLM
        answer = get_answer(query, best_image)

        return search_message, best_image, answer

    # Action for search button
    search_button.click(
        search_with_llm,
        inputs=[query, embeds, imgs],
        outputs=[message2, output_img, output_text]
    )

if __name__ == "__main__":
    demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://293fbc8d93221ef26b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


100%|██████████| 3/3 [00:22<00:00,  7.36s/it]
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-exp:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3953.66ms
